In [43]:
import numpy as np
import pandas as pd

import wrangle as w

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, \
    confusion_matrix,\
    classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
readmes = w.wrangle_github_repositories()

In [3]:
readmes.head(3)

,repo,language,readme_contents,cleaned_readme_contents
0,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...,copyright 2020 huggingface team right reserved...
1,apachecn/ailearning,Python,"<p align=""center"">\n <a href=""https://www.a...",p aligncenter hrefhttpswwwapachecnorg img widt...
2,google-research/bert,Python,"# BERT\n\n**\*\*\*\*\* New March 11th, 2020: S...",bert new march 11th 2020 smaller bert model re...


In [7]:
# remove null rows (adding this to prep stage)
readmes = readmes[readmes.cleaned_readme_contents.isna() == False]

## Split data

In [8]:
train, validate, test = w.train_split(readmes)

In [9]:
readmes.shape, train.shape, validate.shape, test.shape

((493, 4), (275, 4), (119, 4), (99, 4))

In [13]:
X_train = train.cleaned_readme_contents
y_train = train.language
X_validate = validate.cleaned_readme_contents
y_validate = validate.language
X_test = test.cleaned_readme_contents
y_test = test.language

## Baseline

In [19]:
train.language.value_counts(normalize=True)

Python    0.610909
HTML      0.389091
Name: language, dtype: float64

In [31]:
train.language.value_counts(normalize=True)[0]

0.610909090909091

In [20]:
baseline = 'Python'

In [36]:
accuracy = train.language.value_counts(normalize=True)[0]
acc_validate = validate.language.value_counts(normalize=True)[0]
metric_df = pd.DataFrame(data=[
    {
        'model' : 'baseline',
        'accuracy' : accuracy,
        'acc-validate' : acc_validate,
        'difference' : acc_validate - accuracy,
    }
])

In [37]:
metric_df

,model,accuracy,acc-validate,difference
0,baseline,0.610909,0.605042,-0.005867


## The baseline is 'Python' with an accuacy of 61%

# Modeling

In [14]:
# make a Count Vectorizer object
cv = CountVectorizer()
# use the thing
X_bow = cv.fit_transform(X_train)
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.8836363636363637

In [11]:
bag_of_words_cv

<275x36883 sparse matrix of type '<class 'numpy.int64'>'
	with 85429 stored elements in Compressed Sparse Row format>

In [12]:
# make a Tfidf Vectorizer object
tfidf = TfidfVectorizer()
# use the thing
bag_of_words_tfidf = tfidf.fit_transform(X_train)

In [ ]:
cv = CountVectorizer(ngram_range=(1, 3))
bag_of_grams = cv.fit_transform(X_train)

In [ ]:
tfidf = CountVectorizer(ngram_range=(2, 3))
bag_of_grams = tfidf.fit_transform(X_train)

In [41]:
for i in range(1,4):
    for j in range(3,8):
        cv = CountVectorizer(ngram_range=(i, i))
        X_bow = cv.fit_transform(X_train)
        X_val_bow = cv.transform(X_validate)
        tree = DecisionTreeClassifier(max_depth=j)
        tree.fit(X_bow, y_train)
        accuracy = tree.score(X_bow, y_train)
        acc_validate = tree.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'decistion_tree-cv_{i}gram_{j}depth',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

/var/folders/49/v6sf5vy165dghytk4qv6jfhc0000gn/T/ipykernel_1383/941713660.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(
/var/folders/49/v6sf5vy165dghytk4qv6jfhc0000gn/T/ipykernel_1383/941713660.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(
/var/folders/49/v6sf5vy165dghytk4qv6jfhc0000gn/T/ipykernel_1383/941713660.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(
/var/folders/49/v6sf5vy165dghytk4qv6jfhc0000gn/T/ipykernel_1383/941713660.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(
/var/folders/49/

In [42]:
metric_df

,model,accuracy,acc-validate,difference
0,baseline,0.610909,0.605042,-0.005867
1,decistion_tree-cv_1gram_3depth,0.883636,0.789916,-0.093720
2,decistion_tree-cv_1gram_4depth,0.923636,0.764706,-0.158930
3,decistion_tree-cv_1gram_5depth,0.949091,0.747899,-0.201192
4,decistion_tree-cv_1gram_6depth,0.963636,0.764706,-0.198930
5,decistion_tree-cv_1gram_7depth,0.970909,0.722689,-0.248220
6,decistion_tree-cv_2gram_3depth,0.792727,0.697479,-0.095248
7,decistion_tree-cv_2gram_4depth,0.825455,0.697479,-0.127976
8,decistion_tree-cv_2gram_5depth,0.854545,0.697479,-0.157066
9,decistion_tree-cv_2gram_6depth,0.865455,0.689076,-0.176379


In [ ]:
def model_metrics(model,
                  model_name,
                  X_train, 
                  y_train, 
                  X_val, 
                  y_val, 
                  scores=metric_df):
    '''
    model_metrics will use an sklearn model object to 
    create predictions after fitting on our training set, and add
    the model scores to a pre-established metric_df
    returns: metric_df
    '''
    # fit our model object
    model.fit(X_train, y_train['Infant.Mortality'])
    in_sample_pred = model.predict(X_train)
    out_sample_pred = model.predict(X_val)
#     model_name = input('Name for model? python identifer friendly pls.\n')
    y_train[model_name] = in_sample_pred
    y_val[model_name] = out_sample_pred
    print(y_val.shape)
    print(out_sample_pred.shape)
    rmse_val = mean_squared_error(
    y_val['Infant.Mortality'], out_sample_pred, squared=False)
    r_squared_val = explained_variance_score(
        y_val['Infant.Mortality'], out_sample_pred)
    return metric_df.append({
        'model': model_name,
        'rmse': rmse_val,
        'r^2': r_squared_val
    
    }, ignore_index=True)